# Playing with generator

In [1]:
#%%timeit
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import numpy as np
import sys
import copy
from pathlib import Path
#from src.envs.generators import RandomRecurrence
#from src.envs.encoders import Equation, IntegerSeries, RealSeries
from sympy import *
import pickle
from collections import defaultdict, OrderedDict
import math


%load_ext autoreload
%autoreload 2
#%cd ~/recur/notebooks
sys.path.append('/private/home/pakamienny/Research_2/recur')
import src
from src.envs.generators import RandomRecurrence
from src.envs.encoders import  Equation, FloatSequences, IntegerSequences
from src.envs.recurrence import RecurrenceEnvironment


In [3]:
class params:
    float_precision = 3
    max_len = 30
    min_len=10
    max_degree = 6
    max_number = 1e100
    float_sequences = False
    output_numeric=False
    max_exponent = 100
    init_scale = 10
    prob_const=1/3
    prob_n = 1/3
    max_int = 10
    prob_rand = 0.05
    dimension=1
    int_base=10000
    n_predictions=5
    float_tolerance=0.1
    more_tolerance="0.1"
    max_ops=10
    max_token_len=200
    eval_noise=False
    eval_input_length_modulo = -1
    operators_to_remove=""
    required_operators=""
    extra_unary_operators=""
    extra_binary_operators=""
    extra_constants=""
    float_constants=None
    
generator = RandomRecurrence(params)
env = RecurrenceEnvironment(params)
env.rng = np.random.RandomState(0)
import time 

SPECIAL_WORDS = ["EOS", "PAD", "(", ")", "SPECIAL"]

if params.float_sequences:
    input_encoder = FloatSequences(params)
else:
    input_encoder = IntegerSequences(params)
input_words = SPECIAL_WORDS+sorted(list(set(input_encoder.symbols)))

if params.output_numeric:
    if params.float_sequences:
        output_encoder = FloatSequences(params)
    else:
        output_encoder = IntegerSequences(params)
else:
    output_encoder = Equation(params)
print(env.output_word2id)
print(env.output_id2word)

deb = time.time()
k_trials=1000
successful_trials=0
size_input=[]
size_output=[]
infos_stats = {}
#n_ops_stats = {i:0 for i in range(1,(params.max_ops+1)*params.dimension)}

for k in range(k_trials):
    
    x, y, tree, infos = env.gen_expr(False, params.eval_input_length_modulo)
    if tree is not None:
        for info in infos:
            if info not in infos_stats:
                infos_stats[info]=np.zeros((1000,))
            for val in infos[info]:
                infos_stats[info][val]+=1

        successful_trials+=1
        size_input.append(len(x[0]))
        size_output.append(len(y[0]))

size_input=np.array(size_input)
size_output=np.array(size_output)
#n_valid_equations = sum(list(n_ops_stats.values()))
for info in infos_stats:
    non_null_points = np.where(infos_stats[info]!=0.)[0]
    values_non_null_points = infos_stats[info][non_null_points]
    count = values_non_null_points.sum()
    infos_stats[info] = np.concatenate([np.expand_dims(non_null_points,-1), np.expand_dims(values_non_null_points/count,-1)],-1)

print("Percentage of successful generation: {}".format(successful_trials/k_trials*100))
print("Time: {}".format(time.time()-deb))
print(infos_stats)
print("Average sizes: input {}, output {}".format(size_input.max(), size_output.max()))


{'EOS': 0, 'PAD': 1, '(': 2, ')': 3, 'SPECIAL': 4, 'OOD_unary_op': 5, 'OOD_binary_op': 6, 'OOD_constant': 7, '-1': 8, '-10': 9, '-2': 10, '-3': 11, '-4': 12, '-5': 13, '-6': 14, '-7': 15, '-8': 16, '-9': 17, '1': 18, '10': 19, '2': 20, '3': 21, '4': 22, '5': 23, '6': 24, '7': 25, '8': 26, '9': 27, 'abs': 28, 'add': 29, 'idiv': 30, 'mod': 31, 'mul': 32, 'n': 33, 'rand': 34, 'sign': 35, 'sqr': 36, 'step': 37, 'sub': 38, 'x_0_0': 39, 'x_0_1': 40, 'x_0_2': 41, 'x_0_3': 42, 'x_0_4': 43, 'x_0_5': 44, 'x_0_6': 45, '|': 46, '': 7}
{0: 'EOS', 1: 'PAD', 2: '(', 3: ')', 4: 'SPECIAL', 5: 'OOD_unary_op', 6: 'OOD_binary_op', 7: 'OOD_constant', 8: '-1', 9: '-10', 10: '-2', 11: '-3', 12: '-4', 13: '-5', 14: '-6', 15: '-7', 16: '-8', 17: '-9', 18: '1', 19: '10', 20: '2', 21: '3', 22: '4', 23: '5', 24: '6', 25: '7', 26: '8', 27: '9', 28: 'abs', 29: 'add', 30: 'idiv', 31: 'mod', 32: 'mul', 33: 'n', 34: 'rand', 35: 'sign', 36: 'sqr', 37: 'step', 38: 'sub', 39: 'x_0_0', 40: 'x_0_1', 41: 'x_0_2', 42: 'x_0_3

In [ ]:
python train.py --export_data True --dump_path=/private/home/pakamienny/Research_2/recur_datasets/all_domain/ --cpu True --max_epoch 2 --eval_input_length_modulo 5 --epoch_size 10000 

In [ ]:
x = np.array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 566.
             ])
np.argwhere(x!=0.)

# Checking encoder

In [ ]:
class params:
    int_base = 10
    has_separator = False
    
encoder = Equation(params)
print(tree.infix())
encoded = encoder.encode(tree)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded.infix())

In [ ]:
class params:
    a=1
    
from src.envs.encoders import RealSeries

encoder = RealSeries(params)
x = series
print(x)
encoded = encoder.encode(x)
print(encoded)
decoded = encoder.decode(encoded)
print(decoded)